## Supervised Learning
## Project: Finding Donors for *CharityML*

## Background

Udacity Machine Learning Engineer Project. The dataset for this project originates from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Census+Income). The datset was donated by Ron Kohavi and Barry Becker, after being published in the article _"Scaling Up the Accuracy of Naive-Bayes Classifiers: A Decision-Tree Hybrid"_. You can find the article by Ron Kohavi [online](https://www.aaai.org/Papers/KDD/1996/KDD96-033.pdf). The data we investigate here consists of small changes to the original dataset, such as removing the `'fnlwgt'` feature and records with missing or ill-formatted entries.

----
## Exploring the Data

`'income'` is the target label (whether an individual makes more than, or at most, $50,000 annually). All other columns are features about each individual in the census database.

In [ ]:
import numpy as np
import pandas as pd
from time import time
from IPython.display import display

import visuals as vs

%matplotlib inline

data = pd.read_csv("census.csv")

#display(data.head(n = 1))
display(data.head())

### Initial Data Exploration

In [ ]:
n_records = data.shape[0]

#print('Value counts: ', data.income.value_counts())
n_greater_50k = len(data[data['income'] != '<=50K'])

n_at_most_50k = len(data[data.iloc[:,-1]=='<=50K' ])

greater_percent = (n_greater_50k/n_records)

print("Total number of records: {}".format(n_records))
print("Individuals making more than $50,000: {}".format(n_greater_50k))
print("Individuals making at most $50,000: {}".format(n_at_most_50k))
print("Percentage of individuals making more than $50,000: {}%".format(greater_percent))

** Featureset Exploration **

* **age**: continuous. 
* **workclass**: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked. 
* **education**: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool. 
* **education-num**: continuous. 
* **marital-status**: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse. 
* **occupation**: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces. 
* **relationship**: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried. 
* **race**: Black, White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other. 
* **sex**: Female, Male. 
* **capital-gain**: continuous. 
* **capital-loss**: continuous. 
* **hours-per-week**: continuous. 
* **native-country**: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

----
## Preparing the Data

### Finding and transforming Skewed Continuous Features

In [ ]:
# Split the data into features and target label
income_raw = data['income']
features_raw = data.drop('income', axis = 1)

# Visualize skewed continuous features of original data
vs.distribution(data)

Since `'capital-gain'` and `'capital-loss'`, are highly skewed, apply a logarithmic transformation

In [ ]:
# Log-transform the skewed features
skewed = ['capital-gain', 'capital-loss']
features_log_transformed = pd.DataFrame(data = features_raw)
features_log_transformed[skewed] = features_raw[skewed].apply(lambda x: np.log(x + 1))

# Visualize the new log distributions
vs.distribution(features_log_transformed, transformed = True)

### Normalizing Numerical Features

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Initialize a scaler, then apply it to the features
scaler = MinMaxScaler() # default=(0, 1)
numerical = ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

features_log_minmax_transform = pd.DataFrame(data = features_log_transformed)
features_log_minmax_transform[numerical] = scaler.fit_transform(features_log_transformed[numerical])

# Show example of a record with scaling applied
display(features_log_minmax_transform.head(n = 5))

### Implementation: Data Preprocessing

Perform one-hot encoding on the `'features_log_minmax_transform'` data.
 - Convert the target label `'income_raw'` to numerical entries.
   - Set records with "<=50K" to `0` and records with ">50K" to `1`.

In [ ]:
features_final = pd.get_dummies(features_log_minmax_transform)

income = (income_raw == ">50K").astype('int')
#print("Income values: ", income.value_counts())

# Print the number of features after one-hot encoding
encoded = list(features_final.columns)
print("{} total features after one-hot encoding.".format(len(encoded)))

### Shuffle and Split Data
Now all _categorical variables_ have been converted into numerical features, and all numerical features have been normalized. As always, we will now split the data (both features and their labels) into training and test sets. 80% of the data will be used for training and 20% for testing.

In [ ]:
from sklearn.model_selection import train_test_split

# Split the 'features' and 'income' data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_final, 
                                                    income, 
                                                    test_size = 0.2, 
                                                    random_state = 0)

# Show the results of the split
print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))

----
## Evaluating Model Performance
In this section, we will investigate four different algorithms. Three of these algorithms are supervised learners, and the fourth algorithm is a naive predictor.

### Metrics and the Naive Predictor
We are particularly interested in predicting who makes more than \$50,000 accurately. It would seem that using **accuracy** as a metric for evaluating a particular model's performace would be appropriate. Additionally, identifying someone that *does not* make more than \$50,000 as someone who does would be detrimental. 

Therefore, a model's ability to precisely predict those that make more than \$50,000 is *more important* than the model's ability to **recall** those individuals. We can use **F-beta score**:

$$ F_{\beta} = (1 + \beta^2) \cdot \frac{precision \cdot recall}{\left( \beta^2 \cdot precision \right) + recall} $$

In particular, when $\beta = 0.5$, more emphasis is placed on precision. This is the F-score (for simplicity).

For classification problems that are skewed in their classification distributions, like this, accuracy by itself is not a very good metric. Instead, we combine precision and recall to get the F1 score.

### Question 1 - Naive Predictor Performace
* If we chose a model that always predicted an individual made more than $50,000, what would  that model's accuracy and F-score be on this dataset?
The purpose of generating a naive predictor is simply to show what a base model without any intelligence would look like.

In [ ]:
'''
TP = np.sum(income) # Counting the ones as this is the naive case. Note that 'income' is the 'income_raw' data 
encoded to numerical values done in the data preprocessing step.
FP = income.count() - TP # Specific to the naive case

TN = 0 # No predicted negatives in the naive case
FN = 0 # No predicted negatives in the naive case
'''
# Calculate accuracy, precision and recall
accuracy = np.sum(income)/income.count()
recall = np.sum(income)/(np.sum(income) + 0 )
precision = np.sum(income)/(np.sum(income)+ income.count() - np.sum(income))

# Calculate F-score using the formula above for beta = 0.5 and correct values for precision and recall.
fscore = (1+(0.5)**2)*((precision*recall)/((0.5)**2*precision + recall))

# Print the results 
print("Naive Predictor: [Accuracy score: {:.4f}, F-score: {:.4f}]".format(accuracy, fscore))

###  Supervised Learning Models
The following are the supervised learning models that we will employ:
- AdaBoost, 
- Random Forest
- Support Vector Machines (SVM)

For each model chosen

- Describe one real-world application in industry where the model can be applied. 
- What are the strengths of the model; when does it perform well?
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

### 1.  Random Forest Classifier
- This model has been used in analysing and classifying urban space for use for particular activities: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6567884/
- The strengths of this model is that it is easy to use, it works well when implemented as an ensemble method, and it avoids the overfitting of the standalone Decision tree model by incorporating randomness and being implemented in ensemble. It also deals well with unbalanced data sets and can take in both categorical and numerical data
- The weaknesses of this model is that it is not very good for predictions as it produces a discrete classification, and it can overfit if given very noisy data https://blog.citizennet.com/blog/2012/11/10/random-forests-ensembles-and-performance-metrics#:~:text=Random%20Forest%20weaknesses%20are%20that,upon%20your%20own%20data%20set.
- This model is good for our data because we are performing a binaristic classification problem, and Random Forests perform well in such problems and with categorical data.

### 2. AdaBoost Classifier
- This model is used often for image recognition: https://sefiks.com/2018/11/02/a-step-by-step-adaboost-example/, http://acta.uni-obuda.hu/Markoski_Ivankovic_Ratgeber_Pecev_Glusac_57.pdf
- The strengths of this model are that it is flexible and can be used for many different kinds of classifications. Also, it is fast and easy to implement and does not need many parameter settings in order to work effectively. 
- The weaknesses of this model are that it is sensitive to outliers in data and noisy data, and so is sensitive to whether data has been pre-processed or not. 
- AdaBoost is useful here because it works fast and effectively in categorisation problems with large datasets, and deals well with binaristic classification.

### 3. Support Vector Machines
- SVMs are used widely in life sciences models and predictions, especially in telling whether one state exists or not, see: https://link.springer.com/protocol/10.1007/978-1-60327-241-4_13#citeas (examples: https://link.springer.com/article/10.1023/A:1012487302797 https://clincancerres.aacrjournals.org/content/24/22/5574.abstract; https://academic.oup.com/bioinformatics/article/16/10/906/223564)
- The strengths of this classifier include that it works well when there is a relatively high margin of separation and, when this is the case, overfitting is less likely. They are very useful in prediction as they can provide highly accurate separations of data. Also, they work well with high dimensionality in data and are not very susceptible to the influence of outliers (https://iq.opengenus.org/advantages-of-svm/)
- The weaknesses of SVM methods include that it requires more parameters as input than other methods such as AdaBoost, and it can be highly susceptible to noise. Its performance on very large datasets is also not always very good, but can also easily underperform when there are insufficient training data samples https://dhirajkumarblog.medium.com/top-4-advantages-and-disadvantages-of-support-vector-machine-or-svm-a3c06a2b107#:~:text=SVM%20works%20relatively%20well%20when,than%20the%20number%20of%20samples.
- SVM is useful here because we want accurate predictions, and we can modify the kernel used to find better fits to classify our data. It is also useful as a foil for the other two data classifiers, to check out different performances. 

### Implementation - Creating a Training and Predicting Pipeline

In [ ]:
from sklearn.metrics import accuracy_score, fbeta_score

def train_predict(learner, sample_size, X_train, y_train, X_test, y_test): 
    '''
    inputs:
       - learner: the learning algorithm to be trained and predicted on
       - sample_size: the size of samples (number) to be drawn from training set
       - X_train: features training set
       - y_train: income training set
       - X_test: features testing set
       - y_test: income testing set
    '''
    
    results = {}
    
    start = time()
    learner = learner.fit(X_train[:sample_size], y_train[:sample_size])
    end = time()
    
    # Calculate the training time
    results['train_time'] = end - start
        
    # Get the predictions on the test set(X_test),
    # then get predictions on the first 300 training samples(X_train)
    start = time()
    predictions_test = learner.predict(X_test)
    predictions_train = learner.predict(X_train[:300])
    end = time()
    
#    print('Prediction class counts: ', predictions_train.dtype)
    
    # print("{} trained, {} test samples, {} train samples", learner.__class__.__name__, len(X_test), len(X_train))
    #total prediction time
    results['pred_time'] = end - start
            
    # Compute accuracy on the first 300 training samples which is y_train[:300]
    results['acc_train'] = accuracy_score(y_train[:300], predictions_train)
        
    # Compute accuracy on test set using accuracy_score()
    results['acc_test'] = accuracy_score(y_test, predictions_test)
    
    # Compute F-score on the the first 300 training samples using fbeta_score()
#    print('Computing F train')
    results['f_train'] = fbeta_score(y_train[:300], predictions_train, average='binary', beta=0.5)
        
    # Compute F-score on the test set which is y_test
#    print('Computing F test, length of labels: ', len(y_test))
    results['f_test'] = fbeta_score(y_test, predictions_test, average='binary', beta=0.5)
       
    print("{} trained on {} samples.".format(learner.__class__.__name__, sample_size))

    return results

### Implementation: Initial Model Evaluation

In [ ]:
#Import the three supervised learning models from sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression

# Initialize the three models
clf_A = RandomForestClassifier()
clf_B = AdaBoostClassifier()
clf_C = LinearSVC()

# Calculate the number of samples for 1%, 10%, and 100% of the training data
samples_100 = len(y_train)
samples_10 = int(0.1*samples_100)
samples_1 = int(0.01*samples_100)

# Collect results on the learners
results = {}

#clf_C.fit(X_train[:300], y_train[:300])
#y_test = clf_C.predict(X_test[:10])
#print('Number zeros: ', np.count_nonzero(y_test == 0), ' and ones: ', np.count_nonzero(y_test ==1))

for clf in [clf_A, clf_B, clf_C]:
    clf_name = clf.__class__.__name__
    results[clf_name] = {}
    for i, samples in enumerate([samples_1, samples_10, samples_100]):
        results[clf_name][i] = \
        train_predict(clf, samples, X_train, y_train, X_test, y_test)

# Run metrics visualization for the three supervised learning models chosen
vs.evaluate(results, accuracy, fscore)

----
## Improving Results
In this final section, we choose the *best* model to use on the data. We then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`).

### Choosing the Best Model

In light of the metrics for the training data, the Random Forest model has the highest F-score and accuracy scores for the different increments of training data, and both the other models take longer than Random Forest to execute. When we look at the testing data, however, AdaBoost model consistently has the highest F-Score and accuracy for the increments of testing data, even though its does take significantly longer than any of the other two models. In light of this, the best model may be the Linear SVC Model, which is the 'middle best' throughout both sets of data, and consistently takes the low amounts of time to execute in the testing set. However, if the data set is very large, it would be better to use the AdaBoost model, as this can handle large amounts of data and it perfomed best in terms of accuracy and F-score on the testing data, although it was slower comparatively.

### Describing the Model in Layman's Terms

We will use the AdaBoost model to predict income classes. This model works by agglomerating a lot of single tests into one big 'study' of those executed tests, which then helps it to find the optimal result. This model basically combines the answers from a large number of individual tests, taking in all the individual information and combining it all at the end to arrive at one answer that is much better informed since it uses all of these asnwers to get its outcome. 

What the model does is that it looks at a data set, and then successively allocates different weightings of importance to certain parts of that data set, then it 'notes down' the instances of these different weightings. It concludes by comparing and then combining these different weighted sets to come out to a better answer than if no weightings were used at all. 

### Implementation: Model Tuning
Fine tune the chosen model. We use grid search (`GridSearchCV`) and tune parameters. We use the entire training set for this.

In [ ]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV

# Initialize the classifier
clf = AdaBoostClassifier()

#Create the parameters list, using a dictionary if needed.

parameters = {"n_estimators": [50, 100, 200]}
#              "learning_rate": [0.001, 0.005, .01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 1, 2, 10, 20]}

#make an fbeta_score scoring object using make_scorer()
scorer = make_scorer(fbeta_score, beta=0.5)

# Perform grid search on the classifier using GridSearchCV()
grid_obj = GridSearchCV(clf, parameters, scoring=scorer)

#Fit the grid search object to the training data and find the optimal parameters using fit()
grid_fit = grid_obj.fit(X_train, y_train)

# Get the estimator
best_clf = grid_fit.best_estimator_

# Make predictions using the unoptimized and model
predictions = (clf.fit(X_train, y_train)).predict(X_test)
best_predictions = best_clf.predict(X_test)

# Report the before-and-afterscores
print("Unoptimized model\n------")
print("Accuracy score on testing data: {:.4f}".format(accuracy_score(y_test, predictions)))
print("F-score on testing data: {:.4f}".format(fbeta_score(y_test, predictions, beta = 0.5)))
print("\nOptimized Model\n------")
print("Final accuracy score on the testing data: {:.4f}".format(accuracy_score(y_test, best_predictions)))
print("Final F-score on the testing data: {:.4f}".format(fbeta_score(y_test, best_predictions, beta = 0.5)))

### Final Model Evaluation


#### Results:

|     Metric     | Unoptimized Model | Optimized Model |
| :------------: | :---------------: | :-------------: | 
| Accuracy Score |          0.8576   |   0.8651        |
| F-score        |          0.7246   |   0.7396        |


The scores are better in the optimised model, but only marginally. These results from the optimised model are significantly better than the results from the naive predictor benchmarks, much higher on both the accuracy and f-scores. 

----
## Feature Importance

### Implementation - Extracting Feature Importance
We use a supervised learning algorithm that has a `feature_importance_` attribute to the training set and use this to determine the top 5 most important features for the census dataset.

In [ ]:
# Train the supervised model that has 'feature_importances_' on the training set
model = AdaBoostClassifier().fit(X_train,y_train)

# Extract the feature importances
importances = model.feature_importances_

# Plot
vs.feature_plot(importances, X_train, y_train)

###  Extracting Feature Importance

The final feature importance ranking confirms my predictions about the importantce of age, number of years in education and capital gain as factors of great importance in prediction. However, I had thought that both education and capital gain were more important than age, while the actual ordering is the reverse. Clearly age is of greater importance, and capital gain is a more significant predictor than number of years in education. 

Features that I did not think would be as important are capital loss and hours per week. It is clear that people who can engage in capital 'trades', and therefore who would actually be exposed to levels of capital loss, are much more likely to be high-earning individuals, and so it makes sense that this is an important predictor. I had not thought that hours per week would be quite as influential as they are, as I presumed that, after a certain level of income, actual number of hours worked counts much less than the occupation itself, but I can see that perhaps what is happpening is that type of occupation levels out after a certain amount of experience, and the amount of hours worked becomes more important.

### Feature Selection
How does a model perform if we only use a subset of all the available features in the data?

In [ ]:
# Import functionality for cloning a model
from sklearn.base import clone

# Reduce the feature space
X_train_reduced = X_train[X_train.columns.values[(np.argsort(importances)[::-1])[:5]]]
X_test_reduced = X_test[X_test.columns.values[(np.argsort(importances)[::-1])[:5]]]

# Train on the "best" model found from grid search earlier
clf = (clone(best_clf)).fit(X_train_reduced, y_train)

# Make new predictions
reduced_predictions = clf.predict(X_test_reduced)

# Report scores from the final model using both versions of data
print("Final Model trained on full data\n------")
print("Accuracy on testing data: {:.4f}".format(accuracy_score(y_test, best_predictions)))
print("F-score on testing data: {:.4f}".format(fbeta_score(y_test, best_predictions, beta = 0.5)))
print("\nFinal Model trained on reduced data\n------")
print("Accuracy on testing data: {:.4f}".format(accuracy_score(y_test, reduced_predictions)))
print("F-score on testing data: {:.4f}".format(fbeta_score(y_test, reduced_predictions, beta = 0.5)))

### Effects of Feature Selection

The final model on reduced data differs from the full data score by ~3% on accuracy, and around ~6% in the f-scores. These are not huge, but can be significant if the nonprofit concerned really wants to minimise resources spent on reaching out to people, as this number could turn into quite a lot of people if selected from a particularly large set. If time were a factor, I might consider using instead the LinearSVC model or the Random Forest model and checking their final scores in comparison to these, as they both work faster than AdaBoost and might be able to provide good results on the full data set in the same time as AdaBoost on the reduced set. But then again, the precision given by AdaBoost is useful, and we might only achieve with other methods the same precision as AdaBoost does on the reduced set. So, because these scores are around 5%, I would be comfortable using the AdaBoost method if time were a factor, as these amounts are significant but not too much to completely derail the campaign. 